### Part 0: Initialisation and saving qHome

In [28]:
import robotic as ry
import time
max_trials = 10
max_vel = 100
max_acc = max_vel

C = ry.Config()
C.addFile(ry.raiPath('Question-01/environment/cargo.g'))
C.addFile(ry.raiPath('Question-01/environment/cargobot.g'))
C.addFile(ry.raiPath('Question-01/environment/maze.g'))

C.view()
qHome = C.getJointState()
print(qHome)

[ 0.   0.   0.   0.  -0.5  0.  -2.   0.   2.  -0.5]


### 1.1 Part A: gripper to cargo handle position KOMO_1

In [29]:
# komo_1 = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)
komo_1 = ry.KOMO()
komo_1.setConfig(C, True)
komo_1.setTiming(1., 20, 5., 0)

# add collision constraints
komo_1.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
komo_1.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)


# move the gripper to the cargo position
komo_1.addObjective(
    times=[],
    feature=ry.FS.positionDiff,
    frames=["l_gripper", "cargo_handle"],
    type=ry.OT.eq,
    scale=[1e2],
)

# komo_1.addObjective(
#     times=[],
#     feature=ry.FS.positionDiff,
#     frames=["l_gripper", "cargo_handle"],
#     type=ry.OT.sos,
#     scale=[1e1]
# )

# align the grippper with the cargo_handle
komo_1.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', 'cargo_handle'], ry.OT.eq, [1e1])
komo_1.addObjective([], ry.FS.scalarProductYZ, ['l_panda_base', 'cargo_handle'], ry.OT.eq, [1e0])

# solve
ret_1 = ry.NLP_Solver(komo_1.nlp(), verbose=0 ) .solve()
q_gripper_to_cargo = komo_1.getPath()

# for t in range(q_gripper_to_cargo.shape[0]):
#     C.setJointState(q_gripper_to_cargo[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

# komo_1.view_play(False, 2)

if ret_1.feasible:
    print("KOMO_1 is feasible")
else:
    print("KOMO_1 is NOT feasible")

KOMO_1 is feasible


### 1.1 Part A: l_panda_base to goal position KOMO_2

In [30]:
C.setJointState(qHome)

komo_2 = ry.KOMO(C, phases=1, slicesPerPhase=20, kOrder=1, enableCollisions=False)
# komo = ry.KOMO()
# komo_2.setConfig(C, True)
# komo_2.setTiming(1., 20, 5., kOrder=1)

# add control objective to stay close to the initial joint configuration
komo_2.addControlObjective([], 0, 1e-1)

# add collision constraints
komo_2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo_2.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

# Add objective to move cargo to goal area
komo_2.addObjective(
    times=[0., 1.],
    feature=ry.FS.positionDiff,
    frames=["l_panda_base", "goal_area"],
    type=ry.OT.sos
)

komo_2.addObjective(
    times=[0., 1.],
    feature=ry.FS.positionDiff,
    frames=["l_gripper", "goal_area"],
    type=ry.OT.sos,
    target=[0, .6, .46]
)


ret_2 = ry.NLP_Solver(komo_2.nlp(), verbose=0 ) .solve()
q_cargobot_to_goal_area = komo_2.getPath()


# for t in range(q_cargobot_to_goal_area.shape[0]):
#     C.setJointState(q_cargobot_to_goal_area[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

if ret_2.feasible:
    print("KOMO_2 is feasible")
else:
    print("KOMO_2 is NOT feasible")

KOMO_2 is feasible


### 1.1 Part A: Visualisation of KOMO_1 and KOMO_2

In [31]:
for t in range(q_gripper_to_cargo.shape[0]):
    C.setJointState(q_gripper_to_cargo[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)
    
for t in range(q_cargobot_to_goal_area.shape[0]):
    C.setJointState(q_cargobot_to_goal_area[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

# C.setJointState(qHome)
# C.view()

if ret_1.feasible:
    print("KOMO_1 is feasible")
else:
    print("KOMO_1 is NOT feasible")

if ret_2.feasible:
    print("KOMO_2 is feasible")
else:
    print("KOMO_2 is NOT feasible")

# komo_1.view_play(False, 1)
# komo_2.view_play(False, 1)

KOMO_1 is feasible
KOMO_2 is feasible


### 1.2 Part B: RRT pathfinding (KOMO_1)

In [32]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 0}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

C.setJointState(qHome)


while (not foundRRTPath and numOfTrials < max_trials):
  rrt = ry.PathFinder()
  rrt.setProblem(C, [qHome], [q_gripper_to_cargo[0]])
  ret_RRT_1 = rrt.solve()
  path_1 = ret_RRT_1.x
  foundRRTPath = ret_RRT_1.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  
if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")

-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_finger2)-(cargo) [64,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_finger2)-(cargo) [64,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_finger2)-(cargo) [64,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_finger2)-(cargo) [64,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_finger2)-(cargo) [64,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_finger2)-(cargo)

### 1.2 Part B: RRT pathfinding (KOMO_2)

In [33]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 0}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

C.setJointState(q_gripper_to_cargo[0])

while (not foundRRTPath and numOfTrials < max_trials):
  rrt = ry.PathFinder()
  rrt.setProblem(C, [q_gripper_to_cargo[0]], [q_cargobot_to_goal_area[0]])
  ret_RRT_2 = rrt.solve()
  path_2 = ret_RRT_2.x
  foundRRTPath = ret_RRT_2.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  

if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")

-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
*** Found path at trial num # 1  ***


### 1.2 Part B: Visualisation of RRT pathfinding (KOMO_1) and (KOMO_2)

In [34]:
# display the path
sleepTime = 0.05
for t in range(0, path_1.shape[0]-1):
    C.setJointState(path_1[t])
    C.view()
    time.sleep(sleepTime)
    
# display the path
for t in range(0, path_2.shape[0]-1):
    C.setJointState(path_2[t])
    C.view()
    time.sleep(sleepTime)

### 1.3 Part C

Move robot from home position to cargo position. Then attach cargo to gripper

In [35]:
C.setJointState(qHome)

for t in range(0, path_1.shape[0]-1):
    C.setJointState(path_1[t])
    C.view()
    time.sleep(sleepTime)
    
C.attach("l_gripper", "cargo_handle")

Now find new RRT path with cargo attached, b/c current path_2 (from cargo position to goal area) would make the cargo hit the walls

In [36]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 1}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

# C.setJointState(q_gripper_to_cargo[0])

while (not foundRRTPath and numOfTrials < max_trials):
  rrt = ry.PathFinder()
  rrt.setProblem(C, [q_gripper_to_cargo[0]], [q_cargobot_to_goal_area[0]])
  ret_RRT_3 = rrt.solve()
  path_3 = ret_RRT_3.x
  foundRRTPath = ret_RRT_3.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  

if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")


*** Found path at trial num # 1  ***
  -- rrt success: queries:2723 tree sizes: 213 169


In [37]:
for t in range(0, path_3.shape[0]-1):
    C.setJointState(path_3[t])
    C.view()
    time.sleep(sleepTime)
    

In [38]:
C.getFrame("cargo_handle").unLink()